<a href="https://colab.research.google.com/github/Jack0karev/konoha/blob/master/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload() # здесь нужно будет загрузить тот файл из каггл API token, главное, чтобы он назвался kaggle.json 

In [0]:
!cp kaggle.json  ~/.kaggle/

In [0]:
!kaggle competitions download -c iad-deep-learning-sentiment

In [0]:
!unzip x_test.txt.zip  x_test.txt
!unzip x_train.txt.zip  x_train.txt
!unzip y_train.csv.zip  y_train.csv

In [0]:
import pandas as pd
import numpy as np

x_test = pd.read_csv("x_test.txt", sep='\r\t',header=None)
x_train = pd.read_csv("x_train.txt", sep='\r\t',header=None)
y_train = pd.read_csv("y_train.csv")

In [0]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from bs4 import BeautifulSoup
import re

from tqdm import tqdm

In [0]:
from keras.utils import to_categorical
import random

from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential

In [0]:
def clean_sentences(df):
    reviews = []

    for sent in tqdm(df[0]):

        review_text = BeautifulSoup(sent).get_text()

        review_text = re.sub("[^a-zA-Z]"," ", review_text)

        words = word_tokenize(review_text.lower())

        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)


In [0]:
train_sentences = clean_sentences(x_train[:################]) # здесь ты задаешь число входных данных их всего там 3600000 советую взять 1000000
test_sentences = clean_sentences(x_test)
print(len(train_sentences))
print(len(test_sentences))

In [0]:
unique_words = set()
len_max = 0

for sent in tqdm(train_sentences):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)

print(len(list(unique_words)))
print(len_max)

In [0]:
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(train_sentences))

X_train = tokenizer.texts_to_sequences(train_sentences)

X_test = tokenizer.texts_to_sequences(test_sentences)


X_train = sequence.pad_sequences(X_train, maxlen=len_max)

X_test = sequence.pad_sequences(X_test, maxlen=len_max)
y = y_train.Probability.values
print(X_train.shape,X_test.shape)

In [0]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 2)
callback = [early_stopping]

In [0]:
model=Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()

In [0]:
history=model.fit(X_train, y[:#################], # здесь ты задаешь число входных данных их всего там 3600000 советую взять 1000000
                  validation_split = 0.1,epochs=6,
                  batch_size=256, verbose=1,
                  callbacks=callback)

In [0]:
data = model.predict(X_test)

In [0]:
!unzip random_prediction.csv.zip random_prediction.csv
df = pd.DataFrame(data)
df1 = pd.read_csv('random_prediction.csv')

In [0]:
import numpy as np
total = df1.Id
total = pd.DataFrame(total)
total['Probability'] = np.around(df[0], decimals=5).astype(np.double)
total.shape

In [0]:
total.to_csv('sentiment1.csv', index = False)
files.download('sentiment1.csv')